In [1]:
import sys
sys.path.append("../src")


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras


import models
import metricsUtils
import dataUtils
import config
import wandb

2025-12-14 10:56:42.519316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-14 10:56:42.519441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-14 10:56:42.553479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-14 10:56:42.623110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def getData(dataset_dir):
    dataset = tf.data.Dataset.load(dataset_dir)
    classes = ['1_Pronacio', '2_Neutralis', '3_Szupinacio']
    num_classes = len(classes)

    # string -> index TF-ben
    lookup_table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            keys=tf.constant(classes),
            values=tf.constant(list(range(num_classes)), dtype=tf.int32),
        ),
        default_value=-1
    )

    def encode_one_hot_tf(x, y):
        y_idx = lookup_table.lookup(y)            # tf.string -> int
        y_one_hot = tf.one_hot(y_idx, depth=num_classes)
        shape = config.TARGET_IMAGE_SIZE + (3,)
        x.set_shape(shape)                  # fix input shape
        return x, y_one_hot

    dataset = dataset.map(encode_one_hot_tf)

    return dataset

In [3]:
dataset = getData(config.DATA_DIR+"/dataset")
train_dataset, val_dataset, test_dataset = dataUtils.split_dataset(dataset, config.BATCH_SIZE, config.TRAINING_SIZE)

2025-12-14 10:56:47.810487: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-14 10:56:47.858752: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-14 10:56:47.858819: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-14 10:56:47.862477: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-14 10:56:47.862558: I external/local_xla/xla/stream_executor

In [4]:
baselineModel = models.baseline_model()

In [5]:
baselineModel = models.baseline_model()
y_true = []
baselinePredictions = []
for x, y in train_dataset:
    for index in range(len(x)):        
        baselinePredictions.append(baselineModel())
        y_true.append(y[index])
y_true = np.array(y_true)
np_baselinePredictions = np.array(baselinePredictions, dtype=np.float32)

In [6]:
evaluator = metricsUtils.Evaluator(['1_Pronacio', '2_Neutralis', '3_Szupinacio'])
evaluator.evaluate(np.argmax(y_true, axis=1), np_baselinePredictions)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


{'accuracy': 0.4252336448598131,
 'precision_macro': 0.14174454828660435,
 'recall_macro': 0.3333333333333333,
 'f1_macro': 0.19890710382513663,
 'confusion_matrix': array([[ 0, 88,  0],
        [ 0, 91,  0],
        [ 0, 35,  0]]),
 'classification_report': {'1_Pronacio': {'precision': 0.0,
   'recall': 0.0,
   'f1-score': 0.0,
   'support': 88.0},
  '2_Neutralis': {'precision': 0.4252336448598131,
   'recall': 1.0,
   'f1-score': 0.5967213114754099,
   'support': 91.0},
  '3_Szupinacio': {'precision': 0.0,
   'recall': 0.0,
   'f1-score': 0.0,
   'support': 35.0},
  'accuracy': 0.4252336448598131,
  'macro avg': {'precision': 0.14174454828660435,
   'recall': 0.3333333333333333,
   'f1-score': 0.19890710382513663,
   'support': 214.0},
  'weighted avg': {'precision': 0.18082365272076165,
   'recall': 0.4252336448598131,
   'f1-score': 0.25374597824421635,
   'support': 214.0}},
 'roc': {'fpr': {0: array([0., 1.]),
   1: array([0., 1.]),
   2: array([0., 1.]),
   'micro': array([0.   

In [ ]:
modelSimpleCNN1 = models.simpleCNNModel()
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=config.MODEL_SAVE_PATH,
                                                    save_best_only=True,
                                                 verbose=1)

In [9]:
class_weights = {
    0: 1.0,
    1: 1.0,
    2: 146/54  # kb. 2.7-szer nagyobb súly
}
modelSimpleCNN1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)



history = modelSimpleCNN1.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[cp_callback],
    verbose=1,
    class_weight=class_weights
)

Epoch 1/100


2025-12-13 17:18:39.082344: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-12-13 17:18:39.796408: I external/local_xla/xla/service/service.cc:168] XLA service 0x766871ba67f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-13 17:18:39.796465: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-12-13 17:18:39.807692: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1765646319.895377   92140 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6/6 [==============================] - ETA: 0s - loss: 1.4073 - accuracy: 0.1562 

KeyboardInterrupt: 

In [ ]:
testModel = models.simpleCNNModel()
testModel.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])
loss, acc = testModel.evaluate(test_dataset, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

3/3 - 1s - loss: 1.1232 - accuracy: 0.4205 - 713ms/epoch - 238ms/step
Untrained model, accuracy: 42.05%


In [ ]:
# Loads the weights
testModel.load_weights(config.MODEL_SAVE_PATH)

# Re-evaluate the model
loss, acc = testModel.evaluate(test_dataset, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

3/3 - 1s - loss: 1.0609 - accuracy: 0.3523 - 537ms/epoch - 179ms/step
Restored model, accuracy: 35.23%


In [7]:

sweep_config = {
    "method": "bayes",   # inkrementális → bayes jobb, mint grid
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "weifht_decay": {
            "values": [0.0, 1e-4, 1e-3]
        },
        "learning_rate": {
            "values": [1e-4, 3e-4, 1e-3]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "conv1_filters": {
            "values": [32, 64, 128]
        },
        "conv2_filters": {
            "values": [32, 64, 128]
        },
        "dense_filters": {
            "values": [64]
        }
    }
}


In [8]:
import keras
from keras.layers import Conv2D, Input, MaxPool2D, GlobalAveragePooling2D, Dense


class_weights = {
    0: 1.0,
    1: 1.0,
    2: 146/54  # kb. 2.7-szer nagyobb súly
}

def build_model(config):
    model = keras.Sequential([
        keras.Input(shape=(128,128,3)),

        Conv2D(config.conv1_filters, 3, activation='relu'),
        MaxPool2D(),

        Conv2D(config.conv2_filters, 3, activation='relu'),
        MaxPool2D(),

        GlobalAveragePooling2D(),

        Dense(config.dense_filters, activation='relu'),

        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            weight_decay=config.weifht_decay,
            learning_rate=config.learning_rate
            ),
        loss=metricsUtils.OrdinalDistanceLoss(),
        metrics=['accuracy']
    )

    return model

In [9]:
metricsUtils.OrdinalDistanceLoss()

In [11]:
def train_sweep():
    wandb.init()
    configWandb = wandb.config

    model = build_model(configWandb)

    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,  # inkrementális → nem 50!
        callbacks=[
            wandb.keras.WandbMetricsLogger(),
            wandb.keras.WandbModelCheckpoint(filepath=config.MODEL_SAVE_PATH,monitor="val_accuracy",save_best_only=True)
            ]
    )


In [12]:
sweep_id = wandb.sweep(sweep_config, project="better-cnn-incremental")
wandb.agent(sweep_id, function=train_sweep, count=10)

Create sweep with ID: skl7nvc9
Sweep URL: https://wandb.ai/vitmma19/better-cnn-incremental/sweeps/skl7nvc9


wandb: Agent Starting Run: 14ib1xvs with config:
wandb: 	batch_size: 32
wandb: 	conv1_filters: 32
wandb: 	conv2_filters: 64
wandb: 	dense_filters: 64
wandb: 	learning_rate: 0.0003
wandb: 	weifht_decay: 0
wandb: Currently logged in as: 0322sagibenedek (vitmma19) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/20


2025-12-14 10:58:27.750189: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-12-14 10:58:30.722042: I external/local_xla/xla/service/service.cc:168] XLA service 0x796fd23b2ee0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-14 10:58:30.722085: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-12-14 10:58:30.769438: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1765709911.123216   30839 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/20
214/214 [==============================] - 4s 12ms/step - loss: 0.5806 - accuracy: 0.4112 - val_loss: 0.7005 - val_accuracy: 0.3030
Epoch 2/20
214/214 [==============================] - 4s 17ms/step - loss: 0.5772 - accuracy: 0.4252 - val_loss: 0.6978 - val_accuracy: 0.3030
Epoch 3/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5761 - accuracy: 0.4252 - val_loss: 0.6978 - val_accuracy: 0.3030
Epoch 4/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5753 - accuracy: 0.4252 - val_loss: 0.6978 - val_accuracy: 0.3030
Epoch 5/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5757 - accuracy: 0.4252 - val_loss: 0.6979 - val_accuracy: 0.3030
Epoch 6/20
214/214 [==============================] - 3s 10ms/step - loss: 0.5753 - accuracy: 0.4252 - val_loss: 0.6973 - val_accuracy: 0.3030
Epoch 7/20
214/214 [==============================] - 3s 12ms/step - loss: 0.5749 - accuracy: 0.4346 - val_loss: 0.6975 - val_accuracy: 0.3030
Ep

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


214/214 [==============================] - 2s 9ms/step - loss: 0.5744 - accuracy: 0.4252 - val_loss: 0.6966 - val_accuracy: 0.3333
Epoch 10/20
214/214 [==============================] - 2s 9ms/step - loss: 0.5744 - accuracy: 0.4206 - val_loss: 0.6979 - val_accuracy: 0.3333
Epoch 11/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5724 - accuracy: 0.4346 - val_loss: 0.6961 - val_accuracy: 0.3030
Epoch 12/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5690 - accuracy: 0.4673 - val_loss: 0.6925 - val_accuracy: 0.2727
Epoch 13/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5650 - accuracy: 0.5000 - val_loss: 0.6893 - val_accuracy: 0.3030
Epoch 14/20
214/214 [==============================] - 2s 8ms/step - loss: 0.5597 - accuracy: 0.4860 - val_loss: 0.6882 - val_accuracy: 0.3030
Epoch 15/20
214/214 [==============================] - 4s 9ms/step - loss: 0.5566 - accuracy: 0.4813 - val_loss: 0.6806 - val_accuracy: 0.2727
Epoch 16/20

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch/accuracy,▁▂▂▂▂▂▃▄▂▂▃▅█▇▇▇▆▅▅▅
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▇▇▇▇▇▇▇▆▅▅▃▃▂▂▁▁▁
epoch/val_accuracy,▅▅▅▅▅▅▅▅██▅▁▅▅▁█████
epoch/val_loss,█▇▇▇▇▇▇▇▇▇▆▅▄▄▁▇▆▇▇▇
epoch/accuracy,0.46729
epoch/epoch,19
epoch/learning_rate,0.0003
epoch/loss,0.54868
epoch/val_accuracy,0.33333
